In [4]:
from aloe.program import AloeProgram
text = """
modeh(10,daughter(+person,-person)).
modeb(10,parent(+person,-person)).
modeb(10,parent(-person,+person)).
modeb(10,female(+person)).
modeb(10,female(-person)).
%
determination(daughter/2,parent/2).
determination(daughter/2,female/1).
%    
set(verbose,1).

:- begin_bg.
person(ann).
person(mary).
person(tom).
person(eve).
person(lucy).

parent(ann,mary).
parent(ann,tom).
parent(tom,eve).
parent(tom,lucy).

female(ann).
female(mary).
female(eve).
female(lucy).
:- end_bg.
    

:- begin_in_pos.
daughter(mary,ann).
daughter(eve,tom).
:- end_in_pos.

:- begin_in_neg.
daughter(tom,ann).
daughter(tom,eve).
:- end_in_neg.
"""
ap = AloeProgram.build_from_text(text)

ap.induce()

female(V2)
female(V3)
female(V1)
female(V0)
parent(V1,V0)
parent(V4,V3)
parent(V4,V2)
[ 0, 1, 2, 2, 1,-1] Clause(k=0):daughter(V0,V1).
[ 1, 0, 2, 1, 1, 0] Clause(k=5):daughter(V0,V1):-parent(V1,V0).
[ 1, 1, 2, 0, 0, 0] Clause(k=4):daughter(V0,V1):-female(V0).
[ 2, 0, 2, 1, 0,-1] Clause(k=7):daughter(V0,V1):-parent(V1,V0),parent(V4,V2).
[ 2, 0, 2, 1, 0,-1] Clause(k=6):daughter(V0,V1):-parent(V1,V0),parent(V4,V3).
[ 1, 1, 2, 2, 0,-2] Clause(k=2):daughter(V0,V1):-female(V3).
[ 1, 1, 2, 2, 0,-2] Clause(k=6):daughter(V0,V1):-parent(V4,V3).
[ 1, 1, 2, 2, 0,-2] Clause(k=1):daughter(V0,V1):-female(V2).
[ 1, 1, 2, 2, 0,-2] Clause(k=7):daughter(V0,V1):-parent(V4,V2).
[ 1, 1, 1, 2,-1,-3] Clause(k=3):daughter(V0,V1):-female(V1).
Example considered: daughter(mary,ann).
Bottom_i daughter(V0,V1):-female(V2),female(V3),female(V1),female(V0),parent(V1,V0),parent(V4,V3),parent(V4,V2).
Clause found [ 1, 0, 2, 1, 1, 0] Clause(k=5):daughter(V0,V1):-parent(V1,V0).


In [2]:
ap.query('parent(X,mary).')

0
Atom parent(X,mary)
parent/2
Candidates {parent(ann,mary).}
Match {parent(ann,mary).}
Clause parent(ann,mary).
Subst {X: ann}
Atoms []
1


[(True, [{X: ann}])]

In [3]:
def fun():
    yield 1
fun()

<generator object fun at 0x000002463549A190>

In [4]:
from aloe.knowledge import MultipleKnowledge, LogicProgram
ap = AloeProgram.build_from_text(text)
ap.knowledge = MultipleKnowledge(LogicProgram(options=ap.options), ap.knowledge, options=ap.options)
print(ap.query('female(mary).'))


0
Atom female(mary)
female/1
Candidates {female(mary).}
Match {female(mary).}
Clause female(mary).
Subst {}
Atoms []
1
[(True, [{}])]


In [2]:
len(None)

TypeError: object of type 'NoneType' has no len()

In [1]:
from aloe.clause import Clause, Predicate, Function, Constant, Variable, Operator
def print_class_in_depth(element):
    print(element.__class__, element)
    if isinstance(element, Operator):
        for t in element:
            print_class_in_depth(t)

atom = Predicate('female', [Constant('mary')])
print_class_in_depth(atom)            

<class 'aloe.clause.Predicate'> female(mary)
<class 'aloe.clause.Constant'> mary


In [10]:
def fun(a, b, **options):
    print(*options)
    
l = [1,2]
fun(*l, h=30, t=50)


h t


In [20]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def red(text):
    return bcolors.WARNING + text + bcolors.ENDC 

message = 'qfq' + red('blabla')+ 'klqmf'
#print(message)
raise Exception(message)


Exception: qfq[93mblabla[0mklqmf

In [5]:
a = {1,2,3}
b = {4,5}
c = {3,4}
a-c

{1, 2}

In [4]:
set.union({1,2},{3})

{1, 2, 3}

In [1]:
def fun():
    print(__name__)
    
fun()

__main__


In [1]:
from aloe.options import Options
class A:
    def __init__(self, options=None):
        self._options = options if options else Options()
        self._temp_options = dict()
        
    def __getattr__(self, attr):
        if attr=='options':
            if attr in self._temp_options:
                return self._temp_options[attr]
            else:
                return self._options, attr)
        else: return super().__getattr__(self)


In [3]:
a = A()
a.options.a

AttributeError: 'Options' object has no attribute 'options'

In [6]:
a.__dir__()

['_options',
 '_temp_options',
 '__module__',
 '__init__',
 '__getattr__',
 '__dict__',
 '__weakref__',
 '__doc__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [ ]:
self.options.i